## Jupyterhub on Kubernetes

This notebook can be used to launch a standard instance of Jupyterhub which includes a proxy server, and an instance of Jupyterhub. 

Notes on the Server: 
`/etc/jupyterhub/config` is where the configuration is for jupyterhub. 



In [ ]:
import sys 
import importlib
import ruamel.yaml
config_file='config/config.yaml'  #The configuration value set above.
path='/home/jovyan/work/admin-tools/' #Update this value if running locally.
sys.path.append(path+"lib/kuberutils") #Adds a local library path.
import kuberutils as ku   #This imports some utilities.
importlib.reload(ku)      #This just reloads the utilities.
cf=ku.initialize(path,config_file)  #This initializes a configuration object.

### Print Configuration
Optionally you can print the configuration and common commands for your desired cluster. You can use this as a reference and copy and paste into the terminal.

In [ ]:
#This will print common commands for your clustl
print(ruamel.yaml.dump(cf, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper))

### Set Jupyterhub Config
This will setup a basic configuration file for installing Jupyterhub. 

In [ ]:
ku.set_jupyterhub_config(cf)

## PLEASE TAKE A MINUTE TO UPDATE YOUR CONFIG FILE.

If you will be keeping this instance for a while, update your config/config.yaml file so that you don't. 

```
#Start of Azure Configuration
jup_rebuild_config: false

```



## Download Helm Repo

In [ ]:
#Add the jupyterhub repo.
print(ku.bash_command('jup_repo',cf))

In [ ]:
#This will install the jupyterhub instance.
print(ku.bash_command('jup_install',cf))

In [ ]:
#This will check on the status of Jupyterhub.
print(ku.bash_command('jup_describe', cf))

In [ ]:
#Get the ip/url. 
#cf=ku.get_jupyterhub_ip(cf)
ku.bash_command('jup_get_ip',cf)

## Update Authorization
When just starting out, the default configuration launches with no password.  While there is still a username and login screen, 


In [ ]:
#This is how we set the authororization
#ku.update_config('auth','jup_github_auth', cf)
#ku.update_config('auth','jup_google_auth', cf)
ku.update_config('auth','jup_dummy_auth', cf)


In [ ]:
#Add Administrators
ku.update_config('admin','jup_admin', cf)

### Update the Singleuser Container
This can allow you to utilize a different container. 


In [ ]:
ku.update_config('singleuser','jup_singleuser', cf)

In [ ]:
#Upgrading Jupyterhub 
print(ku.bash_command('jup_upgrade',cf))

Adding SSL Support via Let's Encrypt
helm repo update 
helm install --name=letsencrypt --namespace=kube-system stable/kube-lego \
             --set config.LEGO_EMAIL=<your-email> \
             --set config.LEGO_URL=https://acme-v01.api.letsencrypt.org/directory

## Cleanup the Installation
This will cleanup the installation, deleting the instance of Jupyterhub.

In [ ]:
#Delete Jupyterhub with Helm.
print(ku.bash_command('jup_delete_instance',cf))

In [ ]:
#Delete Jupyterhub with Helm.
print(ku.bash_command('jup_delete_namespace',cf))